<a href="https://colab.research.google.com/github/Matheus2608/Data-Science/blob/main/Projeto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fazendo segmentação de instâncias de doenças em morangosaa

## Importando dataset do Kaggle de imagens de morangos com doenças
Para isso, eu fiz o download por linha de comando pelo dataset ser pesado. De referência segui os passos desse site: https://www.analyticsvidhya.com/blog/2021/04/how-to-download-kaggle-datasets-using-jupyter-notebook/

Mas para isso é necessário ter uma conta no kaggle, se você não tiver sugiro que entre no link que estou fazendo o download do dataset e faça o download pra sua máquina e depois transfira para o Colab

In [1]:
!pip install opendatasets
import opendatasets as od

In [2]:
od.download("https://www.kaggle.com/datasets/usmanafzaal/strawberry-disease-detection-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: matheusdev
Your Kaggle Key: ··········


100%|██████████| 534M/534M [00:02<00:00, 214MB/s]


## Treinando Yolact ++
## Setup Inicial
- Para utilizar esse modelo, é necessário importar algumas bibliotecas antes


In [3]:
# Cython precisa ser instalado antes de pycocotools
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib

#DCNv2
!pip install torchvision==0.5.0
!pip install torch==1.4.0

## Importando o código da Yolact do github dos autores dela

In [4]:
# Clone the repo
!git clone https://github.com/dbolya/yolact.git

Cloning into 'yolact'...
remote: Enumerating objects: 2936, done.
remote: Total 2936 (delta 0), reused 0 (delta 0), pack-reused 2936
Receiving objects: 100% (2936/2936), 21.20 MiB | 20.39 MiB/s, done.
Resolving deltas: 100% (2002/2002), done.


## DCNv2
É necessário contruir essa biblioteca, **mas** para isso é necessário que o colab esteja rodando no modo GPU

Para isso:
- Clique em Editar > COnfigurações de Notebook
- Coloque GPU como seu hardware e salve


In [5]:
%cd /content/yolact/external/DCNv2
!python setup.py build develop

/content/yolact/external/DCNv2
running build
running build_ext
building '_ext' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/yolact
creating build/temp.linux-x86_64-3.7/content/yolact/external
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src/cpu
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src/cuda
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-dIfpci/python3.7-3.7.13=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-dIfpci/python3.7-3.7.13=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/content/yolact/exte

## Pesos
- Para não demorar muito o treinamento, vou usar pesos de outro modelo ja treinado
- Tentei de quase todos jeitos possíveis usando comando, mas o yolact não funciona.
- É necessário baixar o peso no github e exportar para esse colab

In [ ]:
%cd /content/yolact
%mkdir weights

## Formatando a base de dados

In [7]:
!pip install -U labelme2coco

In [8]:
%cd /content/yolact/data/
%mkdir dataset_formatado

# import functions
from labelme2coco import get_coco_from_labelme_folder, save_json

# set labelme training data directory
labelme_train_folder = "/content/strawberry-disease-detection-dataset/train/"

# set labelme validation data directory
labelme_val_folder = "/content/strawberry-disease-detection-dataset/val/"

# set path for coco json to be saved
export_dir = "/content/"

# create train coco object
train_coco = get_coco_from_labelme_folder(labelme_train_folder)

# export train coco json
save_json(train_coco.json, export_dir+"train.json")

# create val coco object
val_coco = get_coco_from_labelme_folder(labelme_val_folder, coco_category_list=train_coco.json_categories)

# export val coco json
save_json(val_coco.json, export_dir+"val.json")

/content/yolact/data
There are 1450 listed files in folder .


Converting labelme annotations to COCO format: 100%|██████████| 1450/1450 [00:02<00:00, 568.94it/s]


There are 307 listed files in folder .


Converting labelme annotations to COCO format: 100%|██████████| 307/307 [00:00<00:00, 323.21it/s]


In [9]:
# Fazendo diretório para as imagens
%cd /content/
%mkdir train_images valid_images

#Transferindo as imagens dentro do dataset para o direotrio root
%cd /content/strawberry-disease-detection-dataset/train
%mv *.jpg ../../train_images/
%cd ../val/
%mv *.jpg ../../valid_images/
%cd ..


/content
/content/strawberry-disease-detection-dataset/train
/content/strawberry-disease-detection-dataset/val
/content/strawberry-disease-detection-dataset


Agora que o dataset está formatado do jeito que é necessário, é preciso mudar o arquivo config.py da pasta data do yolact

No final dos datasets coloque o seguinte comando:

In [10]:
'''
# ----------------------- DATASETS ----------------------- #
...

my_custom_dataset = dataset_base.copy({
    'name': 'Doença de Morango',

    'train_images': '/content/train_images',
    'train_info':   '/content/train.json',

    'valid_images': '/content/valid_images',
    'valid_info':   '/content/val.json',

    'has_gt': True,
    'class_names': ("Powdery Mildew Leaf", "Leaf Spot", "Angular Leafspot", "Powdery Mildew Fruit", "Gray Mold", "Anthracnose Fruit Rot", "Blossom Blight")
})

# ----------------------- YOLACT v1.0 CONFIGS ----------------------- #
...
yolact_plus_resnet50_config = yolact_plus_base_config.copy({
    'name': 'yolact_plus_resnet50',
    'dataset' :my_custom_dataset,
    'num_classes': 7,
    'backbone': resnet50_dcnv2_backbone.copy({
        'selected_layers': list(range(1, 4)),
        
        'pred_aspect_ratios': [ [[1, 1/2, 2]] ]*5,
        'pred_scales': [[i * 2 ** (j / 3.0) for j in range(3)] for i in [24, 48, 96, 192, 384]],
        'use_pixel_scales': True,
        'preapply_sqrt': False,
        'use_square_anchors': False,
    }),
})
'''



'\n# ----------------------- DATASETS ----------------------- #\n...\n\nmy_custom_dataset = dataset_base.copy({\n    \'name\': \'Doença de Morango\',\n\n    \'train_images\': \'/content/train_images\',\n    \'train_info\':   \'/content/train.json\',\n\n    \'valid_images\': \'/content/valid_images\',\n    \'valid_info\':   \'/content/val.json\',\n\n    \'has_gt\': True,\n    \'class_names\': ("Powdery Mildew Leaf", "Leaf Spot", "Angular Leafspot", "Powdery Mildew Fruit", "Gray Mold", "Anthracnose Fruit Rot", "Blossom Blight")\n})\n\n# ----------------------- YOLACT v1.0 CONFIGS ----------------------- #\n...\nyolact_plus_resnet50_config = yolact_plus_base_config.copy({\n    \'name\': \'yolact_plus_resnet50\',\n    \'dataset\' :my_custom_dataset,\n    \'num_classes\': 7,\n    \'backbone\': resnet50_dcnv2_backbone.copy({\n        \'selected_layers\': list(range(1, 4)),\n        \n        \'pred_aspect_ratios\': [ [[1, 1/2, 2]] ]*5,\n        \'pred_scales\': [[i * 2 ** (j / 3.0) for j in 

In [11]:
%cd /content/yolact

/content/yolact


In [22]:
!python train.py --resume=weights/yolact_plus_resnet50_54_800000.pth --config=yolact_plus_resnet50_config

Traceback (most recent call last):
  File "train.py", line 504, in <module>
    train()
  File "train.py", line 178, in train
    transform=SSDAugmentation(MEANS))
  File "/content/yolact/utils/augmentations.py", line 684, in __init__
    BackboneTransform(cfg.backbone.transform, mean, std, 'BGR')
AttributeError: 'NoneType' object has no attribute 'transform'


## Yolact++
@article{yolact-plus-tpami2020,

  author  = {Daniel Bolya and Chong Zhou and Fanyi Xiao and Yong Jae Lee},

  journal = {IEEE Transactions on Pattern Analysis and Machine Intelligence}, 

  title   = {YOLACT++: Better Real-time Instance Segmentation}, 

  year    = {2020},
  
}

## Dataset
Afzaal, U.; Bhattarai, B.; Pandeya, Y.R.; Lee, J. An Instance Segmentation Model for Strawberry Diseases Based on Mask R-CNN. Sensors 2021, 21, 6565

In [13]:
# %cd /content/
# %rm -rf strawberry-disease-detection-dataset
# %rm -rf yolact

In [14]:
# rm -rf train.json val.json

In [15]:
# rm -rf train_images/ valid_images/ view